In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### [Cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html)

Cross-validation is essential in model development - it allows us to compare the performance of alternative algorithms and different settings for model hyperparameters, *without* making use of the test data. This is very important so that we can obtain an accurate assessment of the final model performance.

`KFold` is a simple way to get the data indices for cross-validation, which we can loop over.

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
diabetes = load_diabetes()

# Using only the first 250 data points
X = diabetes.data[:250]
y = diabetes.target[:250]

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,shuffle=True,random_state=2)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

lm = LinearRegression()

for train, test in kf.split(X):
    print("training set indices:")
    print(train)
    print("test set indices:")
    print(test)
    lm.fit(X[train], y[train])
    y_pred = lm.predict(X[test])
    print("r2 = %.2f" % r2_score(y[test],y_pred))
    print()

If we just want to calculate a metric, there is another convenient function `cross_val_score`.

In [ ]:
from sklearn.model_selection import cross_val_score

# start with the untrained model
lm = LinearRegression()

# a 5-fold cross-validation, scored using r2
score = cross_val_score( lm,X,y,cv=5,scoring='r2' )
print("Cross-validated r2:")
print(score)

We would usually quote the mean score under cross-validation:

In [ ]:
print("mean r2 =", np.mean(score))

The standard deviation of the cross-validation scores is also useful as an estimate of the error compared to the true performance on unseen test data.

In [ ]:
print("sd =", np.std(score))

In addition to the basic *k*-fold cross-validation, there are many alternative procedures that may be suitable depending on the structure of your particular data set. 

For example, there may be definable subgroups within the data that we might want to leave out of training one at a time, to assess how good the predictor is at extrapolating beyond known groups.

### A cross-validated ROC curve

With cross-validation, each element of the training data set is also used in validation - this is a great way to get a robust overview of predictive performance for our methodology. 

In [ ]:
from sklearn.datasets import fetch_openml

steel = fetch_openml(name='steel-plates-fault', version=1, parser='auto')
X, y = steel.data.to_numpy(), steel.target.to_numpy()

In [ ]:
X.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import RocCurveDisplay, auc
from sklearn.model_selection import StratifiedKFold


n_splits = 5

# StratifiedKFold maintains relative proportions of classes in each fold.
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=4)

# We use a penalised logistic regression model
classifier = LogisticRegression(penalty='l2', solver='liblinear')

tprs = []
aucs = []

# A linspace for the FPR values to allow us to calculate mean behaviour
mean_fpr = np.linspace(0, 1, 100)

# Create the axis
fig, ax = plt.subplots(figsize=(6, 6))
for fold, (train, test) in enumerate(cv.split(X, y)):
    classifier.fit(X[train], y[train])
    viz = RocCurveDisplay.from_estimator(
        classifier,
        X[test],
        y[test],
        name=f"ROC fold {fold}",
        # Make the line partially transparent
        alpha=0.3,
        # Make the linewidth small
        lw=1,
        # Plotting onto the existing axis
        ax=ax,
        plot_chance_level=(fold == n_splits - 1),
    )

    # We calculate interpolated values of TPR at the chosen FPR values
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    # Fix the first TPR as 0
    interp_tpr[0] = 0.0
    # Keep the TPR values for this split
    tprs.append(interp_tpr)
    # Keep the AUC value for this split
    aucs.append(viz.roc_auc)

# Now calculate mean TPR over all the splits
mean_tpr = np.mean(tprs, axis=0)

# Fix the last TPR as 1
mean_tpr[-1] = 1.0
# Calculate AUC for the mean ROC
mean_auc = auc(mean_fpr, mean_tpr)
# Calculate std from the individual AUCs
std_auc = np.std(aucs)
# Plot the line for mean ROC
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

# Calculate std of TPR at each FPR (x-axis) value
std_tpr = np.std(tprs, axis=0)
# Use this to shade a region around the mean ROC curve
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

ax.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title=f"Mean ROC curve with variability')",
)
ax.legend(loc="lower right")

plt.show()

### Exercise



Use k-fold cross-validation to estimate the performance of a MLPRegressor on the `wine_quality_white` dataset. Use a single hidden layer.

Does adding a second hidden layer appear to improve performance?

The `ilpd` dataset contains data from 416 liver patient records (class 1) and 167 non liver patient records (class 2). Plot a cross-validated ROC curve for a random forest classifier on these data.

In [ ]:
ilpd = fetch_openml(name='ilpd', version=1, parser='auto')

In [ ]:
ilpd.data.head()

In [ ]:
ilpd.target.head()